In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import docx
from langchain_openai import ChatOpenAI

import weaviate
from dotenv import load_dotenv,find_dotenv
from weaviate.embedded import EmbeddedOptions
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = PyPDFLoader('../data/RobinsonAdvisory.pdf')
contract = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunk = text_splitter.split_documents(contract)

In [3]:
qa_docx_file_path = "../data/Robinson Q&A.docx"


In [4]:
client = weaviate.Client(embedded_options=EmbeddedOptions)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


Started /home/kerod/.cache/weaviate-embedded: process ID 98072


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-02-26T01:39:02+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-02-26T01:39:02+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-02-26T01:39:02+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-02-26T01:39:02+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-02-26T01:39:02+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [5]:
vectorstore = Weaviate.from_documents(client =  client,
                                      documents= chunk,
                                      embedding=OpenAIEmbeddings(),
                                      by_text = False)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
{"level":"info","msg":"Created shard langchain_71cb5189a73a4378aee2282831d4d4f7_7MTrh6cqrPJJ in 3.153765ms","time":"2024-02-26T01:39:25+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-26T01:39:25+03:00","took":318156}
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSin

{"level":"info","msg":"Created shard langchain_f86269016cd140cf8e42252039dbe5bd_iVnhoilocNVF in 3.223115ms","time":"2024-02-26T02:08:21+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-26T02:08:21+03:00","took":365546}
{"level":"info","msg":"Created shard langchain_2e04bf4bc1124cbe896c5941a2e6100e_Vfob5P1Zboth in 2.70213ms","time":"2024-02-26T02:09:36+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-26T02:09:36+03:00","took":330399}
{"level":"info","msg":"Created shard langchain_4a42bbccdc474df7b1d439042c62eee7_Kt3L1fIGFxI0 in 2.792509ms","time":"2024-02-26T02:10:41+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-26T02:10:41+03:00","took":278850}


In [72]:
retriever = vectorstore.as_retriever()


In [74]:
# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define prompt template
prompt_template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


retrieval = RunnableParallel(
    {"context": retriever,  "question": RunnablePassthrough()} 
)

chain = retrieval | prompt | llm | StrOutputParser()

In [76]:

def extract_questions_answers(docx_file):
    questions = []
    answers = []

    doc = Document(docx_file)

    # Iterate through paragraphs
    for i in range(len(doc.paragraphs)):
        text = doc.paragraphs[i].text.strip()
        # Alternate paragraphs are questions and answers
        if text.startswith("Q"):
            question = text.split(": ", 1)[-1].strip()
            questions.append(question)
        elif text.startswith("A"):
            answer = text.split(": ", 1)[-1].strip()
            answers.append(answer)
            
            i += 1
        else:
            i += 1
        
        # if i % 2 == 0:
        #     questions.append(doc.paragraphs[i].text)
        # else:
        #     answers.append(doc.paragraphs[i].text)

    return questions, answers

In [80]:
questions, ground_truth = extract_questions_answers(qa_docx_file_path)

In [81]:
questions

['Who are the parties to the Agreement and what are their defined names?',
 'What is the termination notice?',
 'What are the payments to the Advisor under the Agreement?',
 'Can the Agreement or any of its obligations be assigned?',
 'Who owns the IP?',
 'Is there a non-compete obligation to the Advisor?',
 'Can the Advisor charge for meal time?',
 'In which street does the Advisor live?',
 'Is the Advisor entitled to social benefits?',
 'What happens if the Advisor claims compensation based on employment relationship with the Company?']

In [82]:
ground_truth

['Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)',
 'According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement.',
 'According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance.',
 '1. Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9  the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking.',
 'According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company.',
 'Yes. During the term of engagement with the Company and fo

In [84]:
ground_truths = [[x] for x in ground_truth]

contexts = []
answers = []
for question in questions:
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])
    answers.append(chain.invoke(question))



/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:

In [26]:
from datasets import Dataset

In [33]:
ground_truth = [[x] for x in ground_truths]

In [85]:
# Preparing the dataset
data = {
            "question": questions,
            "answer": answers,
            "contexts": contexts,
            "ground_truth": ground_truth
        }

# Convert dict to dataset
dataset = Dataset.from_dict(data)


In [86]:
dataset[1]

{'question': 'What is the termination notice?',
 'answer': 'The termination notice is not explicitly mentioned in the provided context.',
 'contexts': ['or\ntransferred\nin\nany\nmanner\nby\nAdvisor\nfor\nany \nreason\nwhatsoever .\nThe\nCompany\nmay\nassign\nthe\nAgreement\nto\na\nsuccessor\nof\nall\nor\nsubstantially\nall \nof\nits\nassets\nor\nbusiness,\nprovided\nthe\nassignee\nhas\nassumed\nthe\nCompany’ s\nobligations\nunder\nthis \nAgreement.\n10.\nGoverning\nLaw\nand\nJurisdiction\n:\nThis\nAgreement\nshall\nbe\ngoverned\nby\nthe\nlaws\nof\nthe\nState\nof \nIsrael,\nwithout\ngiving\neffect\nto\nthe\nrules\nrespecting\nconflicts\nof\nlaws.\nThe\nparties\nconsent\nto\nthe \nexclusive\njurisdiction\nand\nvenue\nof\nTel\nAviv\ncourts\nfor\nany\nlawsuit\nfiled\narising\nfrom\nor\nrelating\nto \nthis\nAgreement.\n11.\nNotices\n:\nNotices\nunder\nthis\nAgreement\nshall\nbe\ndelivered\nto\nthe\nparty’ s\nemail\naddress\nas\nfollows: \nCompany:\ninfo@cloudcorp.com\n,\nAdvisor:\njackrobi

In [87]:
print(type(dataset))


<class 'datasets.arrow_dataset.Dataset'>


In [88]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week

In [89]:
df

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,Who are the parties to the Agreement and what ...,The parties to the Agreement are Cloud Investm...,[or\ntransferred\nin\nany\nmanner\nby\nAdvisor...,Cloud Investments Ltd. (“Company”) and Jack Ro...,0.916667,1.0,1.0,0.953996
1,What is the termination notice?,The termination notice is not explicitly menti...,[or\ntransferred\nin\nany\nmanner\nby\nAdvisor...,According to section 4:14 days for convenience...,0.833333,0.0,NaN,0.000000
2,What are the payments to the Advisor under the...,The payments to the Advisor under the Agreemen...,[incurs\nsuch \nexpenses.\nAs\na\ncondition\nt...,According to section 6: 1. Fees of $9 per hour...,0.833333,1.0,0.6,0.979249
3,Can the Agreement or any of its obligations be...,The Agreement can be assigned by the Company t...,[or\ntransferred\nin\nany\nmanner\nby\nAdvisor...,1. Under section 1.1 the Advisor can’t assign ...,1.000000,1.0,1.0,0.942400
4,Who owns the IP?,The IP is fully and exclusively owned by the C...,[it\nto\nany\nthird\nparty \nwithout\nthe\npri...,According to section 4 of the Undertaking (App...,0.916667,1.0,1.0,0.957935
5,Is there a non-compete obligation to the Advisor?,"Yes, there is a non-compete obligation to the ...",[it\nto\nany\nthird\nparty \nwithout\nthe\npri...,Yes. During the term of engagement with the Co...,0.916667,1.0,1.0,1.000000
6,Can the Advisor charge for meal time?,"No, the Advisor cannot charge for meal time as...","[workspace\nfor\nthe\nAdvisor ,\nas\nlong\nas\...","No. See Section 6.1, Billable Hour doesn’t inc...",1.000000,1.0,NaN,1.000000
7,In which street does the Advisor live?,I don't know.,[any\nsub-agents\nor\ndelegates\nin\nconnectio...,"1 Rabin st, Tel Aviv, Israel",0.000000,0.0,NaN,0.000000
8,Is the Advisor entitled to social benefits?,"No, the Advisor is not entitled to social bene...",[his/her\nservices\nto\nCompany\nand/or\nthe \...,"No. According to section 8 of the Agreement, t...",0.500000,0.0,1.0,0.970786
9,What happens if the Advisor claims compensatio...,If the Advisor claims compensation based on an...,[his/her\nservices\nto\nCompany\nand/or\nthe \...,If the Advisor is determined to be an employee...,0.500000,1.0,1.0,0.888912


{"level":"info","msg":"Created shard langchain_fe492b3756dd4fb59a71cbc5f9b78a9d_WdhrtDTGtuaA in 5.353216ms","time":"2024-02-24T15:40:49+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-24T15:40:49+03:00","took":427516}
{"level":"info","msg":"Created shard langchain_ee6767afef8a494c887e2644822a195b_vtPj8DtXNBmC in 3.354668ms","time":"2024-02-24T15:41:00+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-24T15:41:00+03:00","took":358012}
{"level":"info","msg":"Created shard langchain_5124386acc5c469885271d4faa60a42c_715xgLgbOngb in 3.620809ms","time":"2024-02-24T15:41:11+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-24T15:41:11+03:00","took":305647}
{"level

In [92]:
rec_context = df['context_precision'].mean()
rec_recall = df['context_recall'].mean()
rec_faithfulness = df['faithfulness'].mean()
rec_answer_relevancy = df['answer_relevancy'].mean()


In [94]:
print(f"the average score of context precision is{rec_context}")
print(f"the average score of context recall is{rec_recall}")
print(f"the average score of faithfulness is{rec_faithfulness}")
print(f"the average score of answer relevancy is{rec_answer_relevancy}")

the average score of context precision is0.7416666666366666
the average score of context recall is0.7
the average score of faithfulness is0.9428571428571428
the average score of answer relevancy is0.7693278292264379


As you can see from the scores, on average characteSplitter performes better that the rest